<a href="https://colab.research.google.com/github/gwisnu/misc/blob/main/convert_func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
def ptr2point(excel,area):

  if area == 'paringin':
    mitra = 'BUMA'
    sheet = 'COAL BUMA'
    skip = 7
    column_no = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
    nama_field = ['date','class_unit','bdy_plan_weekly','bdy_plan_monthly','unit','seam_detail','seam','series','lp_n','lp_e','lp_z','dp_n','dp_e','dp_z','rom','access','highest','lowest','v_dist','st_dist','hz_dist','truck_count']
  if area  == 'south':
    sheet='COAL BUMA'
    mitra = 'BUMA'
    skip=8
    column_no = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
    nama_field = ['date','class_unit','bdy_plan_weekly','bdy_plan_monthly','unit','seam_detail','seam','series','lp_n','lp_e','lp_z','dp_n','dp_e','dp_z','rom','access','highest','lowest','v_dist','st_dist','hz_dist','truck_count']
  if area  == 'wara':
    sheet='COAL WARA'
    mitra = 'SIS'
    skip=5
    column_no = [0,1,2,3,4,5,6,7,8,9,10,11,12,15,17,18]
    nama_field = ['date','seam','series','bdy_plan','unit','lp_n','lp_e','lp_z','dp_n','dp_e','dp_z','rom','access','v_dist','hz_dist','truck_count']
  if area  == 'central':
    sheet='COAL CT'
    mitra = 'SIS'
    skip=5
    column_no = [0,1,2,4,5,6,7,8,9,10,11,12,13,16,18,51]
    nama_field = ['date','seam','series','unit','lp_n','lp_e','lp_z','bdy_plan','dp_n','dp_e','dp_z','rom','access','v_dist','hz_dist','truck_count']
  if area  == 'north':
    sheet='COAL North IPPKH'
    mitra = 'SIS'
    skip=5
    column_no = [0,1,2,4,5,6,7,8,9,10,11,12,13,16,18,51]
    nama_field = ['date','seam','series','unit','lp_n','lp_e','lp_z','bdy_plan','dp_n','dp_e','dp_z','rom','access','v_dist','hz_dist','truck_count']

  df_ptr = pd.read_excel(excel, sheet, skiprows=skip, usecols=column_no, header=0)
  df_ptr.columns = nama_field
  df_ptr = df_ptr.dropna(subset=['date'])
  df_ptr['week'] = (df_ptr['date']- pd.DateOffset(days=1)).dt.isocalendar().week #add week column
  df_ptr['unit'] = df_ptr.unit.str.strip()
  df_ptr['unit'] = df_ptr.unit.str[:2] + '-' + df_ptr.unit.str[-2:] #remove -00 from unit
  df_ptr.fillna(0,inplace=True)
  df_ptr['vol_hd'] = df_ptr['hz_dist'] * df_ptr['truck_count']
  df_ptr['vol_vd'] = df_ptr['v_dist'] * df_ptr['truck_count']
  df_ptr['hz_dist_mtd'] = df_ptr['vol_hd'].cumsum()/df_ptr['truck_count'].cumsum()
  df_ptr['v_dist_mtd'] = df_ptr['vol_vd'].cumsum()/df_ptr['truck_count'].cumsum()
  if area == 'paringin':
    df_ptr['lat'] = Prg2Lat(df_ptr['lp_e'], df_ptr['lp_n'])
    df_ptr['lon'] = Prg2Lon(df_ptr['lp_e'], df_ptr['lp_n'])
    df_ptr['lat_d'] = Prg2Lat(df_ptr['dp_e'], df_ptr['dp_n'])
    df_ptr['lon_d'] = Prg2Lon(df_ptr['dp_e'], df_ptr['dp_n'])    
  else:
    df_ptr['lat'] = Ttp2Lat(df_ptr['lp_e'], df_ptr['lp_n'])
    df_ptr['lon'] = Ttp2Lon(df_ptr['lp_e'], df_ptr['lp_n'])
    df_ptr['lat_d'] = Ttp2Lat(df_ptr['dp_e'], df_ptr['dp_n'])
    df_ptr['lon_d'] = Ttp2Lon(df_ptr['dp_e'], df_ptr['dp_n'])    
  df_ptr['pit'] = area
  df_ptr['mitra kerja'] = mitra
  last_update = df_ptr['date'].max().strftime('%y%m%d')
  df_ptr[['date','unit','seam','series','lat','lon','lp_z','rom','access','v_dist','hz_dist','truck_count','pit','mitra kerja']].to_csv('coal_loading_'+area+'_'+last_update+'.csv', index=True, index_label='id')
  csvt_txt = "\"Integer\",\"Date\",\"String\",\"String\",\"String\",\"CoordY\",\"CoordX\",\"Real\",\"String\",\"String\",\"Real\",\"Real\",\"Real\",\"String\",\"String\""
  csvt_file = open('coal_loading_'+area+'_'+last_update+'.csvt', "w")
  n = csvt_file.write(csvt_txt)
  csvt_file.close()
  #export dumping point ptr (rom)
  df_ptr[['date','unit','seam','series','lat_d','lon_d','dp_z','rom','access','v_dist','hz_dist','truck_count','pit','mitra kerja']].to_csv('coal_dumping_rom_'+area+'_'+last_update+'.csv', index=True, index_label='id')
  csvt_txt = "\"Integer\",\"Date\",\"String\",\"String\",\"CoordY\",\"CoordX\",\"Real\",\"String\",\"String\",\"Real\",\"Real\",\"Real\",\"String\",\"String\""
  csvt_file = open('coal_dumping_rom_'+area+'_'+last_update+'.csvt', "w")
  n = csvt_file.write(csvt_txt)
  csvt_file.close()
  print('Selesai, silakan cek file untuk download')

In [ ]:
def ob2point(excel,area):

  if area == 'paringin':
    mitra = 'BUMA'
    sheet = 'OB BUMA'
    skip = 7
    column_no = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,19]
    nama_field = ['date','class_unit','matclass','bdy_plan','unit','lp_n','lp_e','lp_z','dp_n','dp_e','dp_z','disposal','access','highest','lowest','v_dist','hz_dist','truck_count']
  if area  == 'south':
    sheet='OB BUMA'
    mitra = 'BUMA'
    skip=8
    column_no = usecols=[0,1,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,20,22]
    nama_field = ['date','class_unit','bdy_plan','matclass','unit','lp_n','lp_e','lp_z','dp_n','dp_e','dp_z','bdy_disposal','disposal','access','highest','lowest','v_dist','hz_dist','truck_count']
  if area  == 'wara':
    sheet='OB Wara'
    mitra = 'SIS'
    skip=5
    column_no = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,15,16]
    nama_field = ['date','bdy_plan','unit','lp_n','lp_e','lp_z','dp_n','dp_e','dp_z','disposal','access','lowest','highest','v_dist','hz_dist','truck_count']
  if area  == 'central':
    sheet='OB CT'
    mitra = 'SIS'
    skip=5
    column_no = [0,2,3,4,5,6,7,8,9,10,11,12,13,14,16,49]
    nama_field = ['date','unit','lp_n','lp_e','lp_z','bdy_plan','dp_n','dp_e','dp_z','disposal','access','lowest','highest','v_dist','hz_dist','truck_count']
  if area  == 'north':
    sheet='OB North IPPKH'
    mitra = 'SIS'
    skip=5
    column_no = [0,2,3,4,5,6,7,8,9,10,11,12,13,14,16,49]
    nama_field = ['date','unit','lp_n','lp_e','lp_z','bdy_plan','dp_n','dp_e','dp_z','disposal','access','lowest','highest','v_dist','hz_dist','truck_count']

  df_ob = pd.read_excel(excel, sheet, skiprows=skip, usecols=column_no, header=0)
  df_ob.columns = nama_field
  df_ob = df_ob.dropna(subset=['date'])
  df_ob['week'] = (df_ob['date']- pd.DateOffset(days=1)).dt.isocalendar().week #add week column
  df_ob['unit'] = df_ob.unit.str.strip()
  df_ob['unit'] = df_ob.unit.str[:2] + '-' + df_ob.unit.str[-2:] #remove -00 from unit
  df_ob.fillna(0,inplace=True)
  df_ob['vol_hd'] = df_ob['hz_dist'] * df_ob['truck_count']
  df_ob['vol_vd'] = df_ob['v_dist'] * df_ob['truck_count']
  df_ob['hz_dist_mtd'] = df_ob['vol_hd'].cumsum()/df_ob['truck_count'].cumsum()
  df_ob['v_dist_mtd'] = df_ob['vol_vd'].cumsum()/df_ob['truck_count'].cumsum()

  if area == 'paringin':
    df_ob['lat'] = Prg2Lat(df_ob['lp_e'], df_ob['lp_n'])
    df_ob['lon'] = Prg2Lon(df_ob['lp_e'], df_ob['lp_n'])
    df_ob['lat_d'] = Prg2Lat(df_ob['dp_e'], df_ob['dp_n'])
    df_ob['lon_d'] = Prg2Lon(df_ob['dp_e'], df_ob['dp_n'])
  else:
    df_ob['lat'] = Ttp2Lat(df_ob['lp_e'], df_ob['lp_n'])
    df_ob['lon'] = Ttp2Lon(df_ob['lp_e'], df_ob['lp_n'])
    df_ob['lat_d'] = Ttp2Lat(df_ob['dp_e'], df_ob['dp_n'])
    df_ob['lon_d'] = Ttp2Lon(df_ob['dp_e'], df_ob['dp_n'])

  df_ob['pit'] = area
  df_ob['mitra kerja'] = mitra
  last_update = df_ob['date'].max().strftime('%y%m%d')
  #export loading point ob
  df_ob[['date','unit','bdy_plan','lat','lon','lp_z','disposal','access','v_dist','hz_dist','truck_count','pit','mitra kerja']].to_csv('ob_loading_'+area+'_'+last_update+'.csv', index=True, index_label='id')
  csvt_txt = "\"Integer\",\"Date\",\"String\",\"String\",\"CoordY\",\"CoordX\",\"Real\",\"String\",\"String\",\"Real\",\"Real\",\"Real\",\"String\",\"String\""
  csvt_file = open('ob_loading_'+area+'_'+last_update+'.csvt', "w")
  n = csvt_file.write(csvt_txt)
  csvt_file.close()
  #export dumping point ob
  df_ob[['date','unit','bdy_plan','lat_d','lon_d','dp_z','disposal','access','v_dist','hz_dist','truck_count','pit','mitra kerja']].to_csv('ob_dumping_'+area+'_'+last_update+'.csv', index=True, index_label='id')
  csvt_txt = "\"Integer\",\"Date\",\"String\",\"String\",\"CoordY\",\"CoordX\",\"Real\",\"String\",\"String\",\"Real\",\"Real\",\"Real\",\"String\",\"String\""
  csvt_file = open('ob_dumping_'+area+'_'+last_update+'.csvt', "w")
  n = csvt_file.write(csvt_txt)
  csvt_file.close()  
  print('Selesai, silakan cek file untuk download')

In [ ]:
!pip install pyproj
import math
from pyproj import Proj

UTM50s = Proj('+proj=utm +zone=50 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs')
#parameter site calibration Tutupan
rot,dx,dy,sf,ox,oy,olocx,olocy = 57.26, -330718.744,-9748020.012, 1.00008025, 334759.446 ,9757090.68,4040.702,9070.668

#parameter site calibration Paringin
rot_p,dx_p,dy_p,sf_p,ox_p,oy_p,olocx_p,olocy_p = 23.05789276,	-331701.558,	-9741692.346,	1.000039197,	332681.896,	9746705.39,	980.338,	5013.044

def Ttp2Lon(easting,northing):
    x1 = (easting-olocx)/sf+olocx
    y1 = (northing-olocy)/sf+olocy
    x2 = x1-dx
    y2 = y1-dy
    x3 = (x2-ox)*math.cos(math.radians(-rot)) - (y2-oy)*math.sin(math.radians(-rot)) + ox
    y3 = (y2-oy)*math.cos(math.radians(-rot)) + (x2-ox)*math.sin(math.radians(-rot)) + oy
    geo = UTM50s(x3,y3,inverse=True)
    return geo[0]
    
def Ttp2Lat(easting,northing):
    x1 = (easting-olocx)/sf+olocx
    y1 = (northing-olocy)/sf+olocy
    x2 = x1-dx
    y2 = y1-dy
    x3 = (x2-ox)*math.cos(math.radians(-rot)) - (y2-oy)*math.sin(math.radians(-rot)) + ox
    y3 = (y2-oy)*math.cos(math.radians(-rot)) + (x2-ox)*math.sin(math.radians(-rot)) + oy
    geo = UTM50s(x3,y3,inverse=True)
    return geo[1]

def Ttp2Geo(easting,northing):
    x1 = (easting-olocx)/sf+olocx
    y1 = (northing-olocy)/sf+olocy
    x2 = x1-dx
    y2 = y1-dy
    x3 = (x2-ox)*math.cos(math.radians(-rot)) - (y2-oy)*math.sin(math.radians(-rot)) + ox
    y3 = (y2-oy)*math.cos(math.radians(-rot)) + (x2-ox)*math.sin(math.radians(-rot)) + oy
    geo = UTM50s(x3,y3,inverse=True)
    return geo

def Prg2Geo(easting,northing):
    x1 = (easting-olocx_p)/sf_p+olocx_p
    y1 = (northing-olocy_p)/sf_p+olocy_p
    x2 = x1-dx_p
    y2 = y1-dy_p
    x3 = (x2-ox_p)*math.cos(math.radians(-rot_p)) - (y2-oy_p)*math.sin(math.radians(-rot_p)) + ox_p
    y3 = (y2-oy_p)*math.cos(math.radians(-rot_p)) + (x2-ox_p)*math.sin(math.radians(-rot_p)) + oy_p
    geo = UTM50s(x3,y3,inverse=True)
    return geo

def Prg2Lon(easting,northing):
    x1 = (easting-olocx_p)/sf_p+olocx_p
    y1 = (northing-olocy_p)/sf_p+olocy_p
    x2 = x1-dx_p
    y2 = y1-dy_p
    x3 = (x2-ox_p)*math.cos(math.radians(-rot_p)) - (y2-oy_p)*math.sin(math.radians(-rot_p)) + ox_p
    y3 = (y2-oy_p)*math.cos(math.radians(-rot_p)) + (x2-ox_p)*math.sin(math.radians(-rot_p)) + oy_p
    geo = UTM50s(x3,y3,inverse=True)
    return geo[0]
    
def Prg2Lat(easting,northing):
    x1 = (easting-olocx_p)/sf_p+olocx_p
    y1 = (northing-olocy_p)/sf_p+olocy_p
    x2 = x1-dx_p
    y2 = y1-dy_p
    x3 = (x2-ox_p)*math.cos(math.radians(-rot_p)) - (y2-oy_p)*math.sin(math.radians(-rot_p)) + ox_p
    y3 = (y2-oy_p)*math.cos(math.radians(-rot_p)) + (x2-ox_p)*math.sin(math.radians(-rot_p)) + oy_p
    geo = UTM50s(x3,y3,inverse=True)
    return geo[1]
def Geo2Prg(easting,northing):
    geo = UTM50s(easting, northing, inverse=False)
    x1 = (geo[0] - ox_p)*math.cos(math.radians(rot_p)) - (geo[1] - oy_p)*math.sin(math.radians(rot_p)) + ox_p
    y1 = (geo[1] - oy_p)*math.cos(math.radians(rot_p)) + (geo[0] - ox_p)*math.sin(math.radians(rot_p)) + oy_p
    x2 = x1 + dx_p
    y2 = y1 + dy_p    
    x3 = (x2 - olocx_p)*sf_p + olocx_p
    y3 = (y2 - olocy_p)*sf_p + olocy_p
    return [x3,y3]

def Geo2Ttp(easting,northing):
    geo = UTM50s(easting, northing, inverse=False)
    x1 = (geo[0]-ox)*math.cos(math.radians(rot)) - (geo[1]-oy)*math.sin(math.radians(rot)) + ox
    y1 = (geo[1]-oy)*math.cos(math.radians(rot)) + (geo[0]-ox)*math.sin(math.radians(rot)) + oy
    x2 = x1+dx
    y2 = y1+dy    
    x3 = (x2-olocx)*sf+olocx
    y3 = (y2-olocy)*sf+olocy
    return [x3,y3]

def dmd2dd_lat(lat,dir):
    dd = round(math.floor(float(lat) / 100) + (float(lat) % 100) / 60, 6)
    if dir == 'S':
      dd = dd * -1
    return dd
def dmd2dd_lon(lon,dir):
    dd = round(math.floor(float(lon) / 100) + (float(lon) % 100) / 60, 6)
    if dir == 'W':
      dd = dd * -1
    return dd      

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
Prg2Lon(5200,100)

115.51288114773563

In [ ]:
def create_line(df, properties, lat1='lp_n', lon1='lp_e', lat2='dp_n', lon2='dp_e'):
    """
    Turn a dataframe containing point data into a geojson formatted python dictionary

    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lon : the name of the column in the dataframe that contains longitude data
    """

    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection','features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature ={'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'LineString',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [ [row[lon1],row[lat1]],[row[lon2],row[lat2]]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]

        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)

    return geojson
    
def nmea2point(file):
  df = pd.read_csv(file, sep='\t',skiprows=1)
  df.columns = ['value']
  GPRMC = df[df['value'].str.contains("GPRMC")]
  GPRMC = GPRMC['value'].str.split(',', expand=True)
  GPRMC = GPRMC[[0,1,2,3,4,5,6,7,8,9]]
  #RMC
  GPRMC.columns = ['talker_id','time','warning','lat','lat_dir','lon','lon_dir','speed','azimuth','date']
  GPRMC['time'] = GPRMC['time'].astype('string')
  GPRMC['date'] = GPRMC['date'].astype('string')
  GPRMC['datetime'] = pd.to_datetime(GPRMC["date"].astype(str) + " " + GPRMC["time"].astype(str), format='%d%m%y %H%M%S.%f')
  GPRMC['latitude'] = GPRMC.apply(lambda x: dmd2dd_lat(x['lat'], x['lat_dir']), axis=1)
  GPRMC['longitude'] = GPRMC.apply(lambda x: dmd2dd_lon(x['lon'], x['lon_dir']), axis=1)
  GPRMC = GPRMC.reset_index(drop=True)
  GPRMC[['datetime','latitude','longitude','azimuth','speed']]
  #GGA GPS
  GPGGA = df[df['value'].str.contains("GPGGA")]
  GPGGA = GPGGA['value'].str.split(',', expand=True)
  GPGGA = GPGGA[[6,8]]
  GPGGA = GPGGA.reset_index(drop=True)
  GPGGA.columns = ['quality_code','hdop']
  GPGGA.loc[(GPGGA.quality_code == '0'),'fix_quality']='Invalid'          
  GPGGA.loc[(GPGGA.quality_code == '1'),'fix_quality']='GNSS fix (SPS)'          
  GPGGA.loc[(GPGGA.quality_code == '2'),'fix_quality']='DGPS fix'          
  GPGGA.loc[(GPGGA.quality_code == '4'),'fix_quality']='Real Time Kinematic'          
  GPGGA.loc[(GPGGA.quality_code == '8'),'fix_quality']='Simulation mode'          
  #DBT Echosounder
  SDDBT = df[df['value'].str.contains("SDDBT")]
  SDDBT = SDDBT['value'].str.split(',', expand=True)
  SDDBT = SDDBT[[3]]
  SDDBT.columns = ['depth']
  SDDBT = SDDBT.reset_index(drop=True)
  #MTW
  SDMTW = df[df['value'].str.contains("SDMTW")]
  SDMTW = SDMTW['value'].str.split(',', expand=True)
  SDMTW = SDMTW[[1]]
  SDMTW.columns = ['temp_celcius']
  SDMTW = SDMTW.reset_index(drop=True)
  #XDR
  SDXDR = df[df['value'].str.contains("SDXDR")]
  SDXDR = SDXDR['value'].str.split(',', expand=True)
  SDXDR.drop(SDXDR.index[SDXDR[3] == 'P'], inplace = True) #remove warning
  SDXDR = SDXDR[[2,6]]
  SDXDR.columns = ['pitch_deg','roll_deg']
  SDXDR = SDXDR.reset_index(drop=True)

  prop1 = pd.merge(GPRMC[['datetime','latitude','longitude','azimuth','speed']],SDMTW, how='left', left_index=True, right_index=True)
  prop2 = pd.merge(SDDBT,SDXDR, how='left', left_index=True, right_index=True)
  prop3 = pd.merge(prop1, prop2,  how='left', left_index=True, right_index=True)
  nmea_out= pd.merge(prop3,GPGGA[['fix_quality','hdop']], how='left', left_index=True, right_index=True)
  #nmea_out.to_csv('nmea_out.csv', index=True, index_label='id')
  nmea_out['depth'] = nmea_out['depth'].astype(float, errors = 'raise')
  nmea_out['azimuth'] = nmea_out['azimuth'].astype(float, errors = 'raise')
  nmea_out['speed'] = nmea_out['speed'].astype(float, errors = 'raise')
  nmea_out['temp_celcius'] = nmea_out['temp_celcius'].astype(float, errors = 'raise')
  nmea_out['pitch_deg'] = nmea_out['pitch_deg'].astype(float, errors = 'raise')
  nmea_out['roll_deg'] = nmea_out['roll_deg'].astype(float, errors = 'raise')
  nmea_out['hdop'] = nmea_out['hdop'].astype(float, errors = 'raise')
  return nmea_out    